##### DTU - 02806 Social data analyse og visualisering F17 

## Project Assignment A

** Note: ** We followed the approach to better display too much than too few information. That way everyone is able to follow each step precisely. If some outputs are too long download the notebook from nbviewer by using the icon at the top right. Then open the notebook with Jupyter and select Cell > All Output > Toggle Scrolling.

### 0. Video Introduction

In [ ]:
# IPython global cell magic
%reset
%matplotlib inline

In [83]:
import numpy as np
import pandas as pd
import geoplotlib as gp
from random import randint
from scipy import stats
from collections import Counter
from __future__ import division
from collections import defaultdict
from collections import OrderedDict
import matplotlib.pyplot as plt
import geoplotlib as gp
import operator
from geoplotlib.utils import BoundingBox
import plotly.plotly as py
import plotly.tools as pt
import plotly.offline as opy
from plotly.graph_objs import *
from plotly import tools

In [2]:
df = pd.read_csv("NYPD_Motor_Vehicle_Collisions_reduced_data.csv", low_memory=False)

In [3]:
# Drop NaN values from Lat and Lon
df = df.dropna(subset=['LATITUDE','LONGITUDE'])

In [4]:
df.head(5)

,Date,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,...,Maximum Humidity,Mean Temperature,Min Temperature,Minimum Humidity,Precipitation,Sea Level Pressure,Snow,Snow Depth,Visibility,Wind Speed
0,2017/03/28,0:00,BRONX,10466,40.894600,-73.861206,"(40.8946, -73.861206)",NaN,0,0,...,100,7,5,89,18.29,1014.0,0.00,0.00,7.0,9
1,2017/03/28,0:00,BROOKLYN,11218,40.654080,-73.977610,"(40.65408, -73.97761)",18 STREET,0,0,...,100,7,5,89,18.29,1014.0,0.00,0.00,7.0,9
2,2017/03/28,0:00,BROOKLYN,11235,40.583847,-73.940590,"(40.583847, -73.94059)",NaN,0,0,...,100,7,5,89,18.29,1014.0,0.00,0.00,7.0,9
3,2017/03/28,0:00,MANHATTAN,10019,40.762770,-73.975590,"(40.76277, -73.97559)",NaN,0,0,...,100,7,5,89,18.29,1014.0,0.00,0.00,7.0,9
4,2017/03/28,0:00,QUEENS,11429,40.713715,-73.731440,"(40.713715, -73.73144)",222 STREET,0,0,...,100,7,5,89,18.29,1014.0,0.00,0.00,7.0,9


### 1. Preliminary Analysis

In [6]:
year_injur_df=df[(df['NUMBER OF PERSONS INJURED']!=0) | (df['NUMBER OF PERSONS KILLED']!=0)]
A=year_injur_df['NUMBER OF PERSONS INJURED'].sum()
B=year_injur_df['NUMBER OF PERSONS KILLED'].sum()
print "Number of people injured since July 2012 :", A
print "Number of people killed since July 2012 :", B


Number of people injured since July 2012 : 204536
Number of people killed since July 2012 : 943


In [26]:
years=sorted(Counter(df.Date.str[0:4]).keys())
months= range(1,13)
injured_list=OrderedDict()
died_list=OrderedDict()
for year in years:
    for month in months:
        injur_per_year=df[(df['NUMBER OF PERSONS INJURED']!=0) & (df['Date'].str[0:4] == year) & (df['Date'].str[5:7] == str(month).zfill(2))]
        if(injur_per_year['NUMBER OF PERSONS INJURED'].sum()!=0):
            key=str(year)+"/"+str(month).zfill(2)
            injured_list[key]=injur_per_year['NUMBER OF PERSONS INJURED'].sum()

for year in years:
    for month in months:
        died_per_year=df[(df['NUMBER OF PERSONS KILED']!=0) & (df['Date'].str[0:4] == year) & (df['Date'].str[5:7] == str(month).zfill(2))]
        if(died_per_year['NUMBER OF PERSONS KILLED'].sum()!=0):
            key=str(year)+"/"+str(month).zfill(2)
            died_list[key]=died_per_year['NUMBER OF PERSONS KILLED'].sum()

In [29]:
#fig = plt.figure(num=None, figsize=(18, 14))
#ax = fig.add_subplot(211)

#ind = np.arange(len(injured_list))  # the x locations for the groups
#width = 0.35       # the width of the bars

#rects1 = ax.bar(ind, injured_list.values(), width, color='r')
#rects2 = ax.bar(ind, died_list.values(), width, color='y')

# add axis labels
#ax.legend(loc=2, fontsize=15)
#ax.set_title("Number of injured/killed per month in last 5 years", fontsize=20)
#ax.set_xlabel('Timeline', fontsize=15)
#ax.set_ylabel('Count', fontsize=15)

# set axis limit
#ax.set_xticks(ind + width / 2)
#ax.set_xticklabels(injured_list.keys())

# show and save plot
#plt.show()
#fig.savefig('pics/die_injure')

#plotly logic@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
opacity = 0.4
injured_bar = Bar(x=injured_list.keys(),y = injured_list.values(),name="Injured",opacity=opacity)
killed_bar = Bar(x=died_list.keys(),y = died_list.values(),name="Killed",opacity=opacity)
data=[injured_bar,killed_bar]
layout = Layout(title="Number of injured/killed in every month in 5 years",
                barmode='stack',
                xaxis=dict(title='Month'),
                yaxis=dict(title='Count'))
fig = Figure(data=data, layout=layout)
py.iplot(fig)

In [30]:
years=sorted(Counter(df.Date.str[0:4]).keys())
dirty_dis=sorted(Counter(df.BOROUGH).keys())
districts = [x for x in dirty_dis if str(x) != 'nan']

index = np.arange(len(years))
bar_width = 0.1
opacity = 0.4


In [34]:
year_total_dict=defaultdict(list)
data=list()
colors = list()
for i in range(len(years)):
    colors.append('%06X' % randint(0, 0xFFFFFF))
i=0
for district in districts:
    for year in years:
        year_df=df.loc[(df['Date'].str[0:4] == year) & (df['BOROUGH']==district)]
        year_total_dict[district].append(len(year_df.index))
    district_bar = Bar(x=years,y = year_total_dict[district],name=district,marker=dict(color=colors[i]),opacity=opacity)
    data.append(district_bar)
    i=i+1



layout = Layout(title="Total of Accidents in each district in last 5 years",
                xaxis=dict(title='Years'),
                yaxis=dict(title='Accident Total'))
fig = Figure(data=data, layout=layout)
py.iplot(fig)

In [86]:
top_dangers_dict=df['LOCATION'].value_counts().nlargest(10).to_dict()
sorted_top_dangers_dict = sorted(top_dangers_dict.items(), key=operator.itemgetter(1),reverse=True)
print "Top 10 (reported)dangerous cross/street in NYC"
for tup in sorted_top_dangers_dict:
    print df.loc[df['LOCATION'] == tup[0], 'ON STREET NAME'].iloc[0], tup[0], tup[1]
##print "Top 10 (reported)dangerous cross/street in NYC" sorted_top_dangers_dict

Top 10 (reported)dangerous cross/street in NYC
TILLARY STREET                   (40.6960346, -73.9845292) 673
EAST 59 STREET                   (40.7606005, -73.9643142) 543
WEST 42 STREET                   (40.7572323, -73.9897922) 485
ATLANTIC AVENUE                  (40.6757357, -73.8968533) 479
PENNSYLVANIA AVENUE              (40.6585778, -73.8906229) 464
KENMARE STREET                   (40.7203211, -73.9940403) 418
QUEENS BOULEVARD                 (40.7334973, -73.8703694) 415
HUNTS POINT AVENUE               (40.820189, -73.8906752) 405
WEST 42 STREET                   (40.7584292, -73.9926328) 391
3 AVENUE                         (40.7602226, -73.967462) 390


In [133]:
import folium
from colour import Color
red = Color("red")
colors = list(red.range_to(Color("green"),10))
NYC_COORDINATES =(40.71, -73.93)
# create empty map zoomed in on San Francisco
map = folium.Map(location=NYC_COORDINATES, zoom_start=12)
i=0
for tup in sorted_top_dangers_dict:
    lat=float(tup[0].split(',')[0][1:])
    lon=float(tup[0].split(',')[1][:-1])
    folium.CircleMarker(location=[lat,lon], radius=15,
                        popup=df.loc[df['LOCATION'] == tup[0], 'ON STREET NAME'].iloc[0], color=colors[i],
                        fill_color=colors[i]).add_to(map)
    i=i+1

map.save('map.html')
map
